In [0]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go


In [26]:
!wget https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-01.csv

--2020-01-15 16:00:36--  https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_2019-01.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.227.227
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.227.227|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 687088084 (655M) [text/csv]
Saving to: ‘yellow_tripdata_2019-01.csv.1’

yellow_tripdata_201 100%[===================>] 655.26M  83.2MB/s    in 7.4s    

2020-01-15 16:00:43 (88.5 MB/s) - ‘yellow_tripdata_2019-01.csv.1’ saved [687088084/687088084]



# Data PreProcessing

In [27]:
data = pd.read_csv('/content/yellow_tripdata_2019-01.csv')
data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1,1.5,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1,2.6,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3,0.0,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5,0.0,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5,0.0,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN


In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7667792 entries, 0 to 7667791
Data columns (total 18 columns):
VendorID                 int64
tpep_pickup_datetime     object
tpep_dropoff_datetime    object
passenger_count          int64
trip_distance            float64
RatecodeID               int64
store_and_fwd_flag       object
PULocationID             int64
DOLocationID             int64
payment_type             int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
congestion_surcharge     float64
dtypes: float64(9), int64(6), object(3)
memory usage: 1.0+ GB


- Check for Missing Value

In [29]:
data.isnull().sum()

VendorID                       0
tpep_pickup_datetime           0
tpep_dropoff_datetime          0
passenger_count                0
trip_distance                  0
RatecodeID                     0
store_and_fwd_flag             0
PULocationID                   0
DOLocationID                   0
payment_type                   0
fare_amount                    0
extra                          0
mta_tax                        0
tip_amount                     0
tolls_amount                   0
improvement_surcharge          0
total_amount                   0
congestion_surcharge     4855978
dtype: int64

- Apart from congestion surchage, there are no missing values. 
- Drop the column becuase over 50% of it's content are missing.

In [0]:
#drop congestion surchage
data = data.drop(['congestion_surcharge'], axis=1)

In [31]:
data.shape

(7667792, 17)

- Over 7 million rows and 17 columns

Handling Datetime Columns

- Convert the drop off and pickup to datetime
- Extract day, date, and time from the datetime columns
- Drop off datetime columns.

In [0]:
data['tpep_dropoff_datetime'] = pd.to_datetime(data['tpep_dropoff_datetime'])
data['tpep_pickup_datetime'] = pd.to_datetime(data['tpep_pickup_datetime'])


In [0]:
#data['dropoff_day'] = data['tpep_dropoff_datetime'].dt.day
data['dropoff_date'] = data['tpep_dropoff_datetime'].dt.date
data['dropoff_time'] = data['tpep_dropoff_datetime'].dt.time

#data['pickup_day'] = data['tpep_pickup_datetime'].dt.day
data['pickup_date'] = data['tpep_pickup_datetime'].dt.date
data['pickup_time'] = data['tpep_pickup_datetime'].dt.time

In [0]:
data = data.drop(data[['tpep_dropoff_datetime']], axis=1)
data = data.drop(data[['tpep_pickup_datetime']], axis=1)

- Handling Outliers
- From the analysis from the analysis notebook. There where outliers in both the trip distance and fare amount.


In [0]:
data.drop(data[data['fare_amount'] > 5000].index, axis=0, inplace=True)
data.drop(data[data['trip_distance'] > 200].index, axis=0, inplace=True)


In [0]:
data.drop(data[(data['tip_amount'] > 0) & (data['trip_distance'] == 0)].index, axis=0, inplace=True)

In [37]:
data.shape

(7648755, 19)

# Feature Engineering and Selection

In [38]:
data.head()

,VendorID,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,dropoff_date,dropoff_time,pickup_date,pickup_time
0,1,1,1.5,1,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,2019-01-01,00:53:20,2019-01-01,00:46:40
1,1,1,2.6,1,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,2019-01-01,01:18:59,2019-01-01,00:59:47
2,2,3,0.0,1,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,2018-12-21,13:52:40,2018-12-21,13:48:30
3,2,5,0.0,1,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,2018-11-28,15:55:45,2018-11-28,15:52:25
4,2,5,0.0,2,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,2018-11-28,15:58:33,2018-11-28,15:56:57


- Check for Cardinality
  - No cardinal column in the data

In [39]:
data.nunique().sort_values()

store_and_fwd_flag           2
VendorID                     3
improvement_surcharge        4
payment_type                 4
RatecodeID                   7
passenger_count             10
mta_tax                     10
extra                       35
pickup_date                 58
dropoff_date                60
DOLocationID               261
PULocationID               263
tolls_amount               880
tip_amount                3223
trip_distance             4385
fare_amount               6102
total_amount             11885
dropoff_time             86400
pickup_time              86400
dtype: int64